In [1]:
import sys
sys.executable

'/home/cjw/anaconda3/envs/py38/bin/python'

In [12]:
import time

import numpy as np
import itk
import glob
import tifffile

from skimage.util import pad
from matplotlib import pyplot as plt

In [ ]:
tc = itk.TileConfiguration[2]()

In [5]:
tc.Parse("TileConfiguration2.txt")

In [ ]:
prefix = "/path/to/SBEMimage/data/tiles/g0000"

In [20]:

def stitch(z):
    tc = itk.TileConfiguration[2]()
    tc.Parse("TileConfiguration2.txt")
    zstr = f'{z:04d}'
    print(zstr)
    images = list()
    
    for t in range(tc.LinearSize()):
        origin = tc.GetTile(t).GetPosition()
        filename = f'{prefix}/{tc.GetTile(t).GetFileName()}'
        filename = filename.replace('xxxx', zstr)
        _xnp = tifffile.imread(filename).astype(np.float32)
        #_x = itk.imread(filename, itk.F)
        _x = itk.GetImageViewFromArray(_xnp)
        _x.SetOrigin(origin)
        images.append(_x)

    
    montage = itk.TileMontage[type(images[0]), itk.F].New()
    montage.SetMontageSize(tc.GetAxisSizes())
    for t in range(tc.LinearSize()):
        montage.SetInputTile(t, images[t])
    montage.Update()

    ctiles = tc
    for t in range(tc.LinearSize()):
        index = tc.LinearIndexToNDIndex(t)
        #print(index)
        reg_tr = montage.GetOutputTransform(index)
        tile = tc.GetTile(t)
        pos = tile.GetPosition()
        for d in range(2):
            pos[d] -= reg_tr.GetOffset()[d]
        #print(pos)
        tile.SetPosition(pos)
        ctiles.SetTile(t, tile)

    rf = itk.TileMergeImageFilter[type(images[0])].New()
    rf.SetMontageSize(tc.GetAxisSizes())
    for t in range(tc.LinearSize()):
        rf.SetInputTile(t, images[t])
        index = tc.LinearIndexToNDIndex(t)
        rf.SetTileTransform(index, montage.GetOutputTransform(index))

    rf.Update()
    
    _x = rf.GetOutput()
    px = (18300 - _x.shape[-1])
    py = (13725 - _x.shape[-2])
    _x = np.pad(_x, ((0, py), (0, px)))
    _x = 255*(_x - _x.min())/(_x.max() - _x.min())
    _x = _x.astype(np.uint8)
    tifffile.imwrite(f'TestData/test{zstr}.tif', _x, imagej=True)
    
    

In [21]:
for z in range(375,376):
    te = time.time()
    stitch(z)
    print(time.time() - te)

0375
7.478502035140991


In [15]:
out = itk.GetArrayFromImage(rf.GetOutput())

In [8]:
itk.TileMergeImageFilter?

In [11]:
!mkdir test

In [24]:
tc.GetAxisSizes()[0]

3

In [22]:
a = tifffile.imread('test/test0375.tif')
a.min(), a.max(), a.std() 

(0, 255, 56.37591415606711)

In [19]:
a.dtype

dtype('uint8')